In [5]:
import os
import pandas as pd
from openai import OpenAI

In [8]:
#OPEN AI KEY

os.environ["OPENAI_API_KEY"] = "sk-proj-SpmYSDGsdl6Ic8T5oFZfT3BlbkFJmw9sNoC03fA8rad6YNyJ"

OPENAI_API_KEY: str = os.environ.get("OPENAI_API_KEY")


In [4]:
occupations_list = [
    'ICT%20sales%20professional',
    'Marketing%20manager',
    'Product%20analyst',
    'Product%20manager',
    'Product%20designer',
    'Business%20Intelligence%20professional',
    'Infrastructure%20engineer',
    'Computer%20Systems%20Analyst',
    'Software%20infrastructure%20architect',
    'Web%20developer',
    'Software%20developer',
    'App%20developer',
    'User%20interface%20designer',
    'Software%20engineer',
    'Software%20architect',
    'Software%20Quality%20Assurance%20Analysts%20and%20Testers',
    'Embedded%20systems%20engineer',
    'Web%20and%20Digital%20Interface%20Designers',
    'Database%20infrastructure%20engineer',
    'Network%20architect',
    'Database%20Administrators',
    'Database%20Architects',
    'Network%20and%20Computer%20Systems%20Administrator',
    'Artificial%20intelligence%20engineer',
    'Machine%20Learning%20engineer',
    'Data%20science%20engineer',
    'Data%20analyst',
    'Data%20scientist',
    'Artificial%20Intelligence%20scientist',
    'Data%20architect',
    'ICT%20security%20specialist',
    'IT%20security%20operations',
    'Information%20Security%20Analysts',
    'Product%20Security%20and%20IT%20Security%20Integration%20Specialist',
    'Product%20risk%20specialist',
    'Security%20architect',
    'Database%20support%20engineer',
    'Data%20center%20operations%20engineer',
    'Support%20systems%20engineer',
    'Computer%20Network%20Support%20Specialists'
]


In [9]:
import pandas as pd
import os
from openai import OpenAI
import re
import os

# Define la función para limpiar y extraer competencias
def extract_competences(x):
    competences = []
    for item in x.split("\n"):
        if ". " in item:
            competence = item.split(". ")[1].strip()
            if competence:
                competences.append(competence)
        elif "- " in item:
            competence = item.split("- ")[1].strip()
            if competence:
                competences.append(competence)
    print(competences)
    return competences

# Carga el archivos
dict_file = "../Golden/Junio/dict_kso_v16-junio-2024.csv"
file_path = '../indeed_competences/indeed_competences_June_Mexico.csv'
output_path = '../Taxonomías/Junio/1W/tax-infocomm-indeed-1w-junio-MEX-2024.csv'


golden = pd.read_csv(dict_file)
setText = set(golden["Text"].unique())
listText = list(golden["Text"])
relations_set = set()

relations_df = pd.DataFrame(columns=['occ', 'text', 'label'])

if os.path.isfile(file_path) and os.path.getsize(file_path) > 1:
    df = pd.read_csv(file_path)
    print("Archivo encontrado, procesando...")
    for index, row in df.iterrows():
        print(f"Procesando fila {index + 1} del dataframe...")
        for item in row['competences_llm'].split("\n"):
            if ". " in item:
                ksa = item.split(". ")[1]
                found = False
                if ksa in setText:
                    ksa_index = listText.index(ksa)
                    _ksao = golden.at[ksa_index, 'Label']
                    record = {'occ': row['Occ'], 'text': ksa, 'label': _ksao}
                    relations_set.add(tuple(record.items()))
                    relations_df = pd.concat([relations_df, pd.DataFrame([record])], ignore_index=True)
                    found = True
                
                if not found:
                    client = OpenAI()
                    # Usar el modelo finetuned para clasificación
                    try:
                        response = client.chat.completions.create(
                            model="ft:gpt-3.5-turbo-0125:personal::9WV7nHBF",
                            messages=[
                                {"role": "system", "content": "Your job is to identify if the text belongs to knowledge, skill, ability or other based on the KSA taxonomy. Ability is different from skill as it is innate and difficult to measure. You cannot teach ability whereas you can teach skill."},
                                {"role": "user", "content": ksa}
                            ]
                        )
                        classification = response.choices[0].message.content
                        
                        parts = classification.split(':')
                        # If there are two parts and the second part is not empty
                        if len(parts) == 2 and parts[1].strip():
                            # Capitalize the first letter of the second part and strip any leading/trailing whitespace
                            capitalized = parts[1].strip().capitalize()
                        else:
                            capitalized = classification

                        # Capitalize and check validity
                        if capitalized in ['Knowledge', 'Skill', 'Ability', 'Other']:
                            record = {'occ': row['Occ'], 'text': ksa, 'label': capitalized}
                            relations_set.add(tuple(record.items()))
                            relations_df = pd.concat([relations_df, pd.DataFrame([record])], ignore_index=True)
                            
                            # Update Golden file
                            new_row = pd.DataFrame({'Label': [capitalized], 'Text': [ksa]})

                            golden = pd.concat([golden, new_row], ignore_index=True)
                            golden.to_csv(dict_file, index=False)
                            golden = pd.read_csv(dict_file)
                    except Exception as e:
                        print(f"Failed to classify {ksa} due to: {e}")
else:
    print("File not found or is empty")

# Guardar los resultados en un CSV
relations_df.to_csv(output_path, index=False)

df = pd.read_csv(output_path)

# Agrupar por 'occ', 'text', y 'label', contar las entradas y resetear el índice
grouped_df = df.groupby(['occ', 'text', 'label']).size().reset_index(name='count')

# Guardar el DataFrame actualizado en un nuevo archivo CSV
grouped_df.to_csv(output_path, index=False)



Archivo encontrado, procesando...
Procesando fila 1 del dataframe...
Procesando fila 2 del dataframe...
Procesando fila 3 del dataframe...
Procesando fila 4 del dataframe...
Procesando fila 5 del dataframe...
Procesando fila 6 del dataframe...
Procesando fila 7 del dataframe...
Procesando fila 8 del dataframe...
Procesando fila 9 del dataframe...
Procesando fila 10 del dataframe...
Procesando fila 11 del dataframe...
Procesando fila 12 del dataframe...
Procesando fila 13 del dataframe...
Procesando fila 14 del dataframe...
Procesando fila 15 del dataframe...
Procesando fila 16 del dataframe...
Procesando fila 17 del dataframe...
Procesando fila 18 del dataframe...
Procesando fila 19 del dataframe...
Procesando fila 20 del dataframe...
Procesando fila 21 del dataframe...
Procesando fila 22 del dataframe...
Procesando fila 23 del dataframe...
Procesando fila 24 del dataframe...
Procesando fila 25 del dataframe...
Procesando fila 26 del dataframe...
Procesando fila 27 del dataframe...
Pro

In [ ]:
grouped_df = relations_df.groupby(['occ', 'text', 'label', 'value']).size().reset_index(name='count')
grouped_df = grouped_df.drop(columns=['value'])
grouped_df

In [21]:
grouped_df.to_csv('tax-infocomm-jsearch-1w-junio-SFCO-2024.csv', index=False)
